In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

local_path = "/home/user/repos/open-r1_olmo/data/HuggingFaceTB/SmolLM2-1.7B-Instruct"

# model_base = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct", device_map="auto")
# model_instruct = AutoModelForCausalLM.from_pretrained("Neelectric/SmolLM2-1.7B-Instruct_SFT", device_map="auto")
model_grpo = AutoModelForCausalLM.from_pretrained(local_path, local_files_only=True, device_map="auto")
tok = AutoTokenizer.from_pretrained(local_path, local_files_only=True)


/home/user/repos/open-r1_olmo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
inputs = tok(["How many positive whole-number divisors does 196 have?"], return_tensors="pt")
streamer = TextStreamer(tok)

conversation = [
    {"role":"user", "content":"How many positive whole-number divisors does 196 have?"},
    # {"role":"assistant", "content":""}
]

inputs = tok.apply_chat_template(conversation, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(model_grpo.device)
templated = tok.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
inputs = tok(
    templated, 
    return_tensors="pt",
    padding="max_length",
    max_length=100
    )
print(type(inputs))
print(inputs)
print(vars(inputs))

# Despite returning the usual output, the streamer will also print the generated text to stdout.
_ = model_grpo.generate(inputs.input_ids.to("cuda"), 
                   streamer=streamer, 
                   max_new_tokens=2000,
                   do_sample=False
                   )


<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     1,  9690,   198,  2683,   359,
           253,  5356,  5646, 11173,  3365,  3511,   308, 34519,    28,  7018,
           411,   407, 19712,  8182,     2,   198,     1,  4093,   198,  2020,
           800,  2731,  2444,    29,  7979,  3044, 40520,  1072,   216,    33,
            41,    38,   457,    47,     2,   198,     1,   520,  9531,   198]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [3]:
# tok.push_to_hub("Neelectric/SmolLM2-1.7B-Instruct_GRPO")

In [ ]:
import json
file_path = "data/Neelectric/SmolLM2-1.7B-Instruct_SFTv00.02/tokenizer_config.json"
with open(file=file_path) as f:
    data = json.load(f)

print(data)

: 